# Gold Layer 테스트 노트북

dbt로 생성된 Gold 테이블 확인용

In [1]:
import sys

sys.path.append("/app")

from src.utils.spark_builder import get_spark_session
from pyspark.sql import functions as F

# Spark 세션 생성
spark = get_spark_session("GDELT_Gold_Test", "spark://spark-master:7077")
print("✅ Spark 세션 생성 완료")

:: loading settings :: url = jar:file:/usr/local/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-3ab6a12d-6cef-43b9-b643-6e1b5b7f5d26;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.367 in central
:: resolution report :: resolve 154ms :: artifacts dl 5ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.12.367 from central in [default]
	io.delta#delta-core_2.12;2.4.0 from central in [default]
	io.delta#delta-storage;2.4.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from

✅ Spark 세션 생성 완료


In [2]:
# Gold Table 읽기 (dbt가 생성한 Delta 테이블)
gold_df = spark.read.format("delta").load(
    "s3a://warehouse/gold/gdelt_microbatch_country_analysis"
)

print(f"📊 Gold 테이블 레코드 수: {gold_df.count()}")
print(f"📋 컬럼 수: {len(gold_df.columns)}")

25/09/01 15:31:54 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
25/09/01 15:31:58 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


📊 Gold 테이블 레코드 수: 226
📋 컬럼 수: 9


In [3]:
# Gold 테이블 스키마 확인
print("📝 Gold 테이블 컬럼:")
for i, col in enumerate(gold_df.columns, 1):
    print(f"{i:2d}. {col}")

print("\n🔍 스키마:")
gold_df.printSchema()

📝 Gold 테이블 컬럼:
 1. event_date
 2. actor1_country_code
 3. event_root_code
 4. total_events
 5. average_tone
 6. min_tone
 7. max_tone
 8. unique_actors
 9. last_processed_time

🔍 스키마:
root
 |-- event_date: integer (nullable = true)
 |-- actor1_country_code: string (nullable = true)
 |-- event_root_code: string (nullable = true)
 |-- total_events: long (nullable = true)
 |-- average_tone: double (nullable = true)
 |-- min_tone: double (nullable = true)
 |-- max_tone: double (nullable = true)
 |-- unique_actors: long (nullable = true)
 |-- last_processed_time: timestamp (nullable = true)



In [4]:
# Gold 데이터 샘플 확인
print("🔍 Gold 테이블 샘플 데이터:")
gold_df.show(10, truncate=False)

🔍 Gold 테이블 샘플 데이터:


+----------+-------------------+---------------+------------+-------------------+-----------------+-----------------+-------------+--------------------------+
|event_date|actor1_country_code|event_root_code|total_events|average_tone       |min_tone         |max_tone         |unique_actors|last_processed_time       |
+----------+-------------------+---------------+------------+-------------------+-----------------+-----------------+-------------+--------------------------+
|20250901  |TUR                |03             |2           |3.73563218390805   |3.73563218390805 |3.73563218390805 |1            |2025-09-01 12:49:21.557723|
|20250901  |EUR                |09             |4           |-2.78745644599303  |-2.78745644599303|-2.78745644599303|1            |2025-09-01 12:49:21.557723|
|20250901  |GBR                |02             |16          |-1.4080786669674614|-3.76383763837639|1.30548302872063 |4            |2025-09-01 12:49:21.557723|
|20250901  |USA                |19            

In [5]:
# 국가별 이벤트 통계
print("📊 국가별 총 이벤트 수 (Top 10):")
gold_df.groupBy("actor1_country_code").agg(
    F.sum("total_events").alias("total_country_events")
).orderBy(F.desc("total_country_events")).show(10)

📊 국가별 총 이벤트 수 (Top 10):
+-------------------+--------------------+
|actor1_country_code|total_country_events|
+-------------------+--------------------+
|                USA|                 540|
|                NGA|                 284|
|                AUS|                 147|
|                CHN|                 139|
|                RUS|                 109|
|                GBR|                 100|
|                PAK|                  61|
|                ISR|                  58|
|                UKR|                  57|
|                IRL|                  54|
+-------------------+--------------------+
only showing top 10 rows



25/08/28 08:31:01 ERROR StandaloneSchedulerBackend: Application has been killed. Reason: Master removed our application: KILLED
25/08/28 08:31:01 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exiting due to error from cluster scheduler: Master removed our application: KILLED
	at org.apache.spark.errors.SparkCoreErrors$.clusterSchedulerError(SparkCoreErrors.scala:291)
	at org.apache.spark.scheduler.TaskSchedulerImpl.error(TaskSchedulerImpl.scala:978)
	at org.apache.spark.scheduler.cluster.StandaloneSchedulerBackend.dead(StandaloneSchedulerBackend.scala:165)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint.markDead(StandaloneAppClient.scala:263)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint$$anonfun$receive$1.applyOrElse(StandaloneAppClient.scala:170)
	at org.apache.spark.rpc.netty.Inbox.$anonfun$process$1(Inbox.scala:115)
	at org.apache.spark.rpc.netty.Inbox.safelyCall(Inbox.scala:213)
	at org.apache.spark.rpc.netty.Inbox.proce

In [ ]:
# 이벤트 타입별 통계
print("📊 이벤트 타입별 총 이벤트 수 (Top 10):")
gold_df.groupBy("event_root_code").agg(
    F.sum("total_events").alias("total_event_type")
).orderBy(F.desc("total_event_type")).show(10)

In [ ]:
# 톤 분석
print("📊 전체 톤 분석:")
gold_df.select(
    F.avg("average_tone").alias("overall_avg_tone"),
    F.min("min_tone").alias("global_min_tone"),
    F.max("max_tone").alias("global_max_tone"),
).show()

In [ ]:
spark.stop()
print("✅ 테스트 완료!")